# Word2vec preprocessing

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']

# Skip-Gram

indices_to_words(x_batch) = ['against', 'early', 'working', 'class']

indices_to_words(labels_batch) = ['used', 'working', 'early', 'radicals']]

```

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [1]:
import numpy as np
from collections import Counter
import itertools

### CBOW:

In [26]:
class CBOWBatcher:
    def __init__(self, words):
        self.words = words
        self.vocab_size = len(words)
        self.remaining_words = self.words
    
        self.index_words = []
        self.word2index = dict()
        self.index2word = dict()
        
    def delete_rare_words(self, min_frequency = 9):
        words_count = Counter(self.words)
        
        sorted_words_count = sorted(words_count.items(), key=lambda pair: pair[1], reverse=True)

        # choose words with frequency > min_frequency
        mask = list(map(lambda x: x[1] > min_frequency, sorted_words_count))
        sorted_words_count = np.array(sorted_words_count)[mask]

        # add __unk__
        remaining_words = list(map(lambda x: x[0], sorted_words_count))
        remaining_words.append("__unk__")
        self.remaining_words = remaining_words
        self.vocab_size = len(remaining_words)
    
    def create_indexing(self):
        # numericalization
        numbers = np.arange(len(self.remaining_words))

        self.word2index = dict(zip(self.remaining_words, numbers))
        self.index2word = dict(zip(numbers, self.remaining_words))
        # __unk__
        for word in self.words:
            if word in self.word2index:
                self.index_words.append(self.word2index[word])
            else:
                self.index_words.append(self.word2index["__unk__"])
        
    def CBOW(self, batch_size = 4, window_size = 2):
        x_batches = []
        labels_batches = [] 
        
        for i in np.arange(window_size, len(self.index_words) - window_size):
            labels_batches.append(self.index_words[i])
            temp_list = self.index_words[i - window_size: i] + self.index_words[i+1: i+window_size+1]
            x_batches.append(temp_list)
        
        x_batches = np.array(x_batches)
        labels_batches = np.array(labels_batches)
        
        if labels_batches.shape[0]%batch_size != 0:
            raise Exception("Incorrect batch_size: {}".format(batch_size))
        
        x_batches = x_batches.reshape(-1, batch_size, 2*window_size)
        labels_batches = labels_batches.reshape(-1, batch_size,)
        
        return x_batches, labels_batches
    
    def indices_to_words(self, batch):
        words_batch = []
        shape = np.array(batch).shape
        batch_flatten = np.array(batch).flatten()
        
        for i in batch_flatten:
            if i in self.index2word:
                words_batch.append(self.index2word[i])
            else:
                raise Exception("Incorrect numericalization: {} in {}".format(i, batch))
                
        words_batch = np.array(words_batch).reshape(shape)
        return words_batch

In [38]:
text_file = open("text8", "r")
words = text_file.read().split(' ')
test = words[:100000]

batcher = CBOWBatcher(test)
batcher.delete_rare_words(2)
batcher.create_indexing()
y = batcher.CBOW(batch_size = 4)

In [45]:
print("x_batches: ", y[0].shape)
print("label_batches: ", y[1].shape)

x_batches:  (24999, 4, 4)
label_batches:  (24999, 4)


The first dimension is a number of batches.

In [40]:
x_batch = y[0][2]
label_batch = y[1][2]
print("x_batch: \n{}\n{}\n".format(x_batch, batcher.indices_to_words(x_batch)))
print("label_batch: \n{}\n{}\n".format(label_batch, batcher.indices_to_words(label_batch)))

x_batch: 
[[  65  104  124  450]
 [ 104  216  450  583]
 [ 216  124  583 4214]
 [ 124  450 4214  137]]
[['first' 'used' 'early' 'working']
 ['used' 'against' 'working' 'class']
 ['against' 'early' 'class' '__unk__']
 ['early' 'working' '__unk__' 'including']]

label_batch: 
[216 124 450 583]
['against' 'early' 'working' 'class']



In [84]:
import gc
gc.collect()

10

#### Conclusion: 

everything seems working.